# Pollen Data Scraper for City of Atlanta using BeautifulSoup

In [ ]:
# Import libraries
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def url_builder(month, year):
    url_base = 'http://www.atlantaallergy.com/pollen_counts/index/'
    if month < 10:
        url = url_base + str(year) + "/0" + str(month)
    else:
        url = url_base + str(year) + "/" + str(month)
    print(url)
    return(url)

In [ ]:
def get_pollen_counts_from_x_to_y(first_year,last_year,first_month,last_month):
    #Collects pollen counts data over range of years and months
    #Iterate over the years
    for y in list(range(first_year,last_year+1)):
        #iterate over the months
        for m in list(range(first_month,last_month+1)):
            time.sleep(1)
            url= url_builder(m,y)
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')
            month_item_list = soup.find(class_='calendar-body')
            week_list = month_item_list.find(class_ = 'calendar-row')
            day_list = month_item_list.find_all(class_ = 'calendar-day')


            for day in day_list:
            #     print(day.find(class_ = 'day-num').contents[0])
            #     print(day.find(class_ = 'day-num').prettify())
            #     print(day.find_all('a'))
                a = day.find_all('a')
                href = a[0].get('href')
                date = href.replace("/pollen_counts/index/", "")
                pcount = list(a[0].children)
                if len(pcount) == 1 :
                    print(date, pcount[0])
                    data.append([date, pcount[0]])
                else:
                    print(date, 'NA')
                    data.append([date, "NA"])
            print(str(m)+"/"+str(y))
    return(data)

In [ ]:
headers = ["date","pcount"]
data = []


early_years_data = get_pollen_counts_from_x_to_y(1992,2018,1,12)
#Add Pollen counts for 2019 up to July
data_2019 = get_pollen_counts_from_x_to_y(2019,2019,1,7)
data = early_years_data + data_2019

In [ ]:
df = pd.DataFrame(data, columns=headers)
#removes duplicates which result from end and beginning of month day overlap
df.drop_duplicates().to_csv("./atlanta_pollen_history.csv")